In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
#% matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

from scipy.integrate import solve_ivp

class Lorenz(object):
    def __init__(self, sigma=10., beta=8 / 3., rho=28., **kwargs):
        super().__init__(**kwargs)

        self.sigma = sigma
        self.beta = beta
        self.rho = rho

    def __call__(self, t, x):
        """ x here is [x, y, z] """
        # x = tf.cast(x, tf.float64)

        dx_dt = self.sigma * (x[1] - x[0])
        dy_dt = x[0] * (self.rho - x[2]) - x[1]
        dz_dt = x[0] * x[1] - self.beta * x[2]

        dX_dt = tf.stack([dx_dt, dy_dt, dz_dt])
        return dX_dt

Jacobian定義

In [2]:
def jacobian(t, f, x):
    """ return jacobian matrix of f at x"""
    n = x.shape[-1].value
    fx = f(t, x)
    if x.shape[-1].value != fx.shape[-1].value:
        print('For calculating Jacobian matrix',
              'dimensions of f(x) and x must be the same')
        return

    return tf.concat([tf.gradients(fx[i], x) for i in range(0, n)], 0)

# 各変数、パラメータ、関数の定義

In [3]:
rho = tf.Variable(28.0, dtype=tf.float64,
                             name='rho')
sigma = tf.Variable(10.0, dtype=tf.float64,
                     name='sigma')
beta = tf.Variable(8.0/3.0, dtype=tf.float64,
                                    name='sigma')

f_lorenz = Lorenz(sigma=sigma, rho=rho, beta=beta)

# define a state variable x as Variable
x = tf.Variable([1,1,10], dtype=tf.float64, name='x') 
fx= f_lorenz(None,x)

# op for initialize x from outside of TF
x_init=  tf.placeholder(dtype=tf.float64, shape=(3), name='x_init')
init_x_op = tf.assign(x,x_init)

Instructions for updating:
Colocations handled automatically by placer.
